# Landsat-8 with Coiled

# Create Dask cluster with Coiled

In [ ]:
import coiled

cluster = coiled.Cluster(n_workers=10, configuration="coiled-examples/pangeo")

In [ ]:
from dask.distributed import Client

client = Client(cluster)
client

# Load Landsat-8 data

In [ ]:
import dask
import xarray as xr
import geopandas as gpd
from satsearch import Search
import intake
import ast
import pandas as pd

def load_landsat8_data():
    # NOTE this STAC API endpoint does not currently search the entire catalog
    bbox = (-124.71, 45.47, -116.78, 48.93) #(west, south, east, north) 

    time_range = '2019-01-01/2020-10-01'

    # STAC metadata properties
    properties = ['eo:row=027', 'eo:column=047', 'landsat:tier=T1'] 
    results = Search.search(collection='landsat-8-l1', 
                            bbox=bbox,
                            datetime=time_range,
                            property=properties,
                            sort=['<datetime'])

    items = results.items()
    items.save('subset.geojson')
    
    gf = gpd.read_file('subset.geojson')
    band_info = pd.DataFrame(ast.literal_eval(gf.iloc[0]['eo:bands']))
    bands = band_info.query('gsd == 30').common_name.to_list()
    
    @dask.delayed
    def stacitem_to_dataset(item):
        stacked = catalog[item.id].stack_bands(bands)
        da = stacked(chunks=dict(band=1, x=8000, y=2048)).to_dask()
        da['band'] = bands # use common names
        da = da.expand_dims(time=[pd.to_datetime(item.datetime)])
        ds = da.to_dataset(dim='band')
        return ds

    catalog = intake.open_stac_item_collection(items)
    lazy_datasets = []
    for i, item in gf.iterrows():
        ds = stacitem_to_dataset(item)
        lazy_datasets.append(ds)
    datasets = dask.compute(*lazy_datasets)
    
    return xr.concat(datasets, dim='time')
    
    
DS = load_landsat8_data()
print(f"Dataset size: {DS.nbytes / 1e9} [Gb]")
DS

# Distributed NDVI computation

Now that we have our `xarray.DataSet` which contains the Landsat-8 dataset, we'll calculate the NDVI with all our data:

In [ ]:
NDVI = (DS['nir'] - DS['red']) / (DS['nir'] + DS['red'])
NDVI

In [ ]:
NDVI.data

In [ ]:
# NDVI = NDVI.persist()

In [ ]:
# # Plotting pulls data from distributed cluster memory on-demand
# NDVI.hvplot.image(groupby='time', x='x',y='y', 
#                   cmap='BrBG', clim=(-1,1),
#                   rasterize=True, dynamic=True, 
#                   width=700, height=500)

We can then calculate and plot the average NDVI over some spatial selection

In [ ]:
NDVI.sel(x=slice(4.5e5, 5.0e5), y=slice(5.25e6, 5.2e6)).mean(dim=['time']).plot.imshow()